In [1]:
import Pkg
Pkg.activate(".")
Pkg.add("Distributions")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("ARFFFiles")
Pkg.add("DataFrames")
Pkg.add("ScikitLearn")
Pkg.add("NPZ")
Pkg.resolve();

  Activating project at `/Volumes/lodygaw/2022/learning/wdum`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving package versions...
  No 

In [ ]:
function load_dataset(name::String, split::String, return_X_y::Bool)
    @assert split in ["train", "test"] """You must specify "train" or "test" in  split parameter!"""
    
    r = Regex("$(name)Dimension[0-9]*_$(uppercase(split)).arff")
    filenames = filter(x->match(r,x)!==nothing, readdir("datasets/$(name)/"))
end

In [2]:
using ARFFFiles, DataFrames
acc_x_train = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension1_TRAIN.arff");
acc_y_train = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension2_TRAIN.arff");
acc_z_train = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension3_TRAIN.arff");
gyr_x_train = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension4_TRAIN.arff");
gyr_y_train = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension5_TRAIN.arff");
gyr_z_train = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension6_TRAIN.arff");
acc_x_test = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension1_TEST.arff");
acc_y_test = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension2_TEST.arff");
acc_z_test = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension3_TEST.arff");
gyr_x_test = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension4_TEST.arff");
gyr_y_test = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension5_TEST.arff");
gyr_z_test = ARFFFiles.load(DataFrame, "./datasets/BasicMotions/BasicMotionsDimension6_TEST.arff");

Stratyfikacja - wylosowanie osobno z każdej klasy, żeby uniknąć przypadkowego podziału danych na zbiór testowy i treningowy, w których będzie brakować jakiś klas.

In [3]:
X_train = Array{Float64,3}(permutedims(cat(Array(acc_x_train[:,1:100]), Array(acc_y_train[:,1:100]), Array(acc_z_train[:,1:100]), Array(gyr_x_train[:,1:100]), Array(gyr_y_train[:,1:100]), Array(gyr_z_train[:,1:100]) ; dims=3),(1,3,2)));
y_train = Array(acc_x_train[:,101]);
X_test = Array{Float64,3}(permutedims(cat(Array(acc_x_test[:,1:100]), Array(acc_y_test[:,1:100]), Array(acc_z_test[:,1:100]), Array(gyr_x_test[:,1:100]), Array(gyr_y_test[:,1:100]), Array(gyr_z_test[:,1:100]) ; dims=3),(1,3,2)));
y_test = Array(acc_x_test[:,101]);

In [4]:
include("rocket.jl")

rocket = Rocket(10_000)
fit!(rocket, X_train);

# using NPZ
# npzwrite("weights.npz", rocket.kernels[1])
# npzwrite("lengths.npz", rocket.kernels[2])
# npzwrite("biases.npz", rocket.kernels[3])
# npzwrite("dilations.npz", rocket.kernels[4])
# npzwrite("paddings.npz", rocket.kernels[5])
# npzwrite("num_channel_indices.npz", rocket.kernels[6])
# npzwrite("channel_indices.npz", rocket.kernels[7])

In [5]:
X_train_transform = transform!(rocket, X_train)
X_test_transform = transform!(rocket, X_test)

40×20000 Matrix{Float32}:
 0.8       12.2711  0.333333  10.6475   …  0.984375  7.28445  0.2   1.88095
 0.457143  21.3142  0.452381  13.8534      0.609375  3.44641  0.6   7.55089
 0.5       11.9676  0.5       10.9609      0.90625   6.7974   0.5   4.88879
 0.528571  19.9806  0.47619   14.9113      0.703125  4.96848  0.6   8.75006
 0.585714  14.1829  0.47619   25.4742      0.625     2.86406  0.5  10.7241
 0.557143  22.0573  0.488095   9.40935  …  0.65625   3.63114  0.4   8.15069
 0.5       21.9326  0.428571  21.5212      0.625     3.45545  0.6   6.56597
 0.542857  17.9263  0.5       13.1234      0.625     3.28033  0.8  12.5698
 0.557143  17.4591  0.488095  14.302       0.625     4.22762  0.4   7.31534
 0.5       25.6478  0.416667  15.66        0.90625   5.85676  0.5   9.39244
 0.428571  22.2117  0.511905  12.6734   …  0.609375  3.88088  0.5   9.68899
 0.442857  22.3716  0.47619   10.0044      0.578125  4.40861  0.6  10.7118
 0.471429  20.0956  0.511905  10.7166      0.578125  3.79991  0.8

In [6]:
using ScikitLearn
@sk_import linear_model : RidgeClassifierCV
alphas = [1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]
ridge = RidgeClassifierCV(alphas=alphas, normalize=true);
ridge.fit(X_train_transform, y_train)
ridge.score(X_test_transform, y_test)

Solving environment: ...working... done

# All requested packages already installed.



┌ Info: Running `conda install -y -c conda-forge llvm-openmp` in root environment
└ @ Conda /Users/wiktor/.julia/packages/Conda/x2UxR/src/Conda.jl:127
/Users/wiktor/.julia/conda/3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alphas to: original_alphas * n_samples. 
  warnings.warn(


1.0